# The step-wise GNN model

First import the needed libaries

In [27]:
import torch
import numpy as np

from torch.nn.utils.rnn import pad_sequence
from grid2op_env.grid_to_gym import Grid_Gym
from evaluation.restore_agent import restore_agent

Initalize the environment. For convenience we take the `env_config` from one of the already trained SAC agents. Only modification is `conn_matrix` being `True`.

In [28]:
env_config = {'act_on_single_substation': True,
            'convert_to_tuple': True,
            'env_name': 'rte_case14_realistic',
            'keep_actions': 
                            ['change_bus'],
            'keep_observations': ['rho',
                                'gen_p',
                                'load_p',
                                'p_or',
                                'p_ex',
                                'timestep_overflow',
                                'maintenance',
                                'topo_vect'],
            'log_reward': False,
            'medha_actions': True,
            'reward_scaling_factor': 3,
            'rho_threshold': 0.9,
            'run_until_threshold': True,
            'scale': True,
            'use_parametric': False,
            'conn_matrix': True
            }

rllib_env = Grid_Gym(env_config);

SUBSTATION NUMBER: 0
Choosing 2 out of 3
Choosing 3 out of 3
SUBSTATION NUMBER: 1
Choosing 3 out of 6
Choosing 4 out of 6
Choosing 5 out of 6
Choosing 6 out of 6
SUBSTATION NUMBER: 2
Choosing 2 out of 4
Choosing 3 out of 4
Choosing 4 out of 4
SUBSTATION NUMBER: 3
Choosing 3 out of 6
Choosing 4 out of 6
Choosing 5 out of 6
Choosing 6 out of 6
SUBSTATION NUMBER: 4
Choosing 3 out of 5
Choosing 4 out of 5
Choosing 5 out of 5
SUBSTATION NUMBER: 5
Choosing 3 out of 6
Choosing 4 out of 6
Choosing 5 out of 6
Choosing 6 out of 6
SUBSTATION NUMBER: 6
Choosing 2 out of 3
Choosing 3 out of 3
SUBSTATION NUMBER: 7
Choosing 1 out of 2
Choosing 2 out of 2
SUBSTATION NUMBER: 8
Choosing 3 out of 5
Choosing 4 out of 5
Choosing 5 out of 5
SUBSTATION NUMBER: 9
Choosing 2 out of 3
Choosing 3 out of 3
SUBSTATION NUMBER: 10
Choosing 2 out of 3
Choosing 3 out of 3
SUBSTATION NUMBER: 11
Choosing 2 out of 3
Choosing 3 out of 3
SUBSTATION NUMBER: 12
Choosing 2 out of 4
Choosing 3 out of 4
Choosing 4 out of 4
SUBS

/opt/miniconda3/envs/rlib_grid/lib/python3.7/site-packages/lightsim2grid/_aux_add_trafo.py:33: UserWarning: There were some Nan in the pp_net.trafo["tap_neutral"], they have been replaced by 0
  warnings.warn("There were some Nan in the pp_net.trafo[\"tap_neutral\"], they have been replaced by 0")
/opt/miniconda3/envs/rlib_grid/lib/python3.7/site-packages/lightsim2grid/_aux_add_trafo.py:41: UserWarning: There were some Nan in the pp_net.trafo["tap_step_percent"], they have been replaced by 0
  warnings.warn("There were some Nan in the pp_net.trafo[\"tap_step_percent\"], they have been replaced by 0")
/opt/miniconda3/envs/rlib_grid/lib/python3.7/site-packages/lightsim2grid/_aux_add_trafo.py:46: UserWarning: There were some Nan in the pp_net.trafo["tap_pos"], they have been replaced by 0
  warnings.warn("There were some Nan in the pp_net.trafo[\"tap_pos\"], they have been replaced by 0")
/opt/miniconda3/envs/rlib_grid/lib/python3.7/site-packages/gym/spaces/box.py:74: UserWarning: WARN: B

Now that we have the environment setup, we need to create *observation converter* that will supply the input to the neural network in a good format. The input consists of:

- node features
- the adjacenecy matrix (ices)

In [29]:
env = rllib_env.org_env

In [30]:
def vectorize_obs(obs, env, hazard_threshold = 0.9):
   """
   Vectorize the gym observation.

   :param OrderedDict obs: gym observation
   :param Grid2Op_environment env: grid2op environment. Used to fetch the 
      ids of different objects.
   :param float hazard_threshold
   """

   length = env.action_space.dim_topo # number of bus bars == number of nodes in the graph

   # rho is symmetric for both ends of the line [56,1]
   rho = torch.zeros(length)
   rho[env.action_space.line_or_pos_topo_vect] = torch.from_numpy(obs["rho"])
   rho[env.action_space.line_ex_pos_topo_vect] = torch.from_numpy(obs["rho"])

   # active power p [56,1]
   p = torch.zeros(length)
   p[env.action_space.gen_pos_topo_vect] = torch.from_numpy(obs["gen_p"]) # generator active production
   p[env.action_space.load_pos_topo_vect] = torch.from_numpy(obs["load_p"]) # load active consumption
   p[env.action_space.line_or_pos_topo_vect] = torch.from_numpy(obs["p_or"]) # origin active flow
   p[env.action_space.line_ex_pos_topo_vect] = torch.from_numpy(obs["p_ex"]) # Extremity active flow

   # overflow [56,1]
   over = torch.zeros(length)
   over[env.action_space.line_or_pos_topo_vect] = torch.from_numpy(obs["timestep_overflow"]).float()
   over[env.action_space.line_ex_pos_topo_vect] = torch.from_numpy(obs["timestep_overflow"]).float()

   # one-hot topo vector [56,3]
   topo_vect_one_hot = torch.zeros(length,3)
   topo_vect = obs["topo_vect"]
   topo_vect[topo_vect==-1] = 0 # change disconneted from -1 to 0
   topo_vect_one_hot = torch.nn.functional.one_hot(torch.from_numpy(topo_vect).to(torch.int64), num_classes=3)

   # powerline maintenance
   # maintenance = torch.zeros(length)
   # maintenance[env.action_space.line_or_pos_topo_vect] = torch.from_numpy(obs["maintenance"]).float()
   # maintenance[env.action_space.line_ex_pos_topo_vect] = torch.from_numpy(obs["maintenance"]).float()

   # manual feature thresholding 
   hazard = torch.zeros(length) # [56,1]
   hazard[env.action_space.line_or_pos_topo_vect] = (torch.from_numpy(obs["rho"]) > hazard_threshold).float()
   hazard[env.action_space.line_ex_pos_topo_vect] = (torch.from_numpy(obs["rho"]) > hazard_threshold).float()

   vectorized_obs = torch.stack([rho,p,over, hazard], dim = 1)
   vectorized_obs = torch.concat([vectorized_obs, topo_vect_one_hot], dim = -1)
   
   return vectorized_obs

In [31]:
vectorize_obs(rllib_env.reset(), env).shape

torch.Size([56, 7])

This way we get the feature vector of the shape `(n_nodes, n_features)`.
What will also be useful down the road is the information on what elements belong to what substation.

In [33]:
elem_to_sub_id_arrays = [env.load_to_subid, env.gen_to_subid, env.line_or_to_subid, env.line_ex_to_subid, env.storage_to_subid]
elem_to_elem_id = [env.reset().load_pos_topo_vect, env.reset().gen_pos_topo_vect, env.reset().line_or_pos_topo_vect, env.reset().line_ex_pos_topo_vect, env.reset().storage_pos_topo_vect]
elem_to_sub_id_arrays, "------ \n", elem_to_elem_id

([array([ 1,  2,  3,  4,  5,  8,  9, 10, 11, 12, 13], dtype=int32),
  array([1, 2, 5, 7, 0], dtype=int32),
  array([ 0,  0,  1,  1,  1,  2,  3,  5,  5,  5,  8,  8,  9, 11, 12,  3,  3,
          4,  6,  8], dtype=int32),
  array([ 1,  4,  2,  3,  4,  3,  4, 10, 11, 12,  9, 13, 10, 12, 13,  6,  8,
          5,  7,  6], dtype=int32),
  array([], dtype=int32)],
 '------ \n',
 [array([ 8, 12, 18, 23, 29, 39, 42, 45, 48, 52, 55], dtype=int32),
  array([ 7, 11, 28, 34,  2], dtype=int32),
  array([ 0,  1,  4,  5,  6, 10, 15, 24, 25, 26, 35, 36, 41, 47, 51, 16, 17,
         22, 31, 38], dtype=int32),
  array([ 3, 19,  9, 13, 20, 14, 21, 43, 46, 49, 40, 53, 44, 50, 54, 30, 37,
         27, 33, 32], dtype=int32),
  array([], dtype=int32)])

In [34]:
from collections import defaultdict, OrderedDict

sub_id_to_elem_id = defaultdict(list) # necessary for pooling over substations

for sub_id_arr, elem_id_arr in zip(elem_to_sub_id_arrays,elem_to_elem_id):
    for sub_id, elem_id in zip(sub_id_arr, elem_id_arr):
        sub_id_to_elem_id[sub_id].append(elem_id)

sub_id_to_elem_id = OrderedDict(sorted(sub_id_to_elem_id.items()))


To check if we have correctly assigned each element to a correct substation we can look at the global `env.sub_info` information.

In [35]:
for sub_id, num_connected in enumerate(env.sub_info):
    if len(sub_id_to_elem_id[sub_id]) != num_connected:
        raise ValueError("Something went wrong for substation {sub_id}".format(sub_id = sub_id))

In [36]:
len(rllib_env.all_actions_dict)

106

## Encoder 
Now we can proceed to programming the encoder.
The encoder will have the following structure:


In [37]:
import torch.nn as nn
import torch.nn.functional as F

from layers.graph_attention_layer import GATLayer

def pool_per_substation(arr, sub_to_id, pooling_operator = "mean"):
    """
    Pool the observations over the substations.
    
    Keyword arguments:
    ----------
    arr: np.array
        The array to pool over
    dim: int
        The dimension to pool over.
    sub_to_id: dict
        The dictionary that maps the substation id to the index of the array.
    pooling_operator: str
        "mean" or "max". Defaults to mean.
    """

    pooled = [] 
    for sub, elements in sub_to_id.items():
        if pooling_operator == "max":
            pooled.append(torch.mean(arr[:, elements, :], dim = 1, keepdim=True))
        else:
            pooled.append(torch.mean(arr[:, elements, :], dim = 1, keepdim=True))
    
    return torch.cat(pooled, dim = 1)
        

class SubstationModel(nn.Module):
    def __init__(self, num_features, hidden_dim, nheads, num_layers = 6, dropout=0):
        """
        Constructs the Encoder.

        Args:
            num_features (int): Number of features of each node.
            hidden_dim ([int]): Number of features of each node after transformation.
            nheads (int): number of attention heads.
            num_layers (int, optional): Number of GAT layers. Defaults to 6.
            dropout (int, optional): Dropout probability. Defaults to 0.
        """
        super(SubstationModel, self).__init__()
        self.linear = nn.Linear(num_features, hidden_dim)
        self.gat_layers = nn.ModuleList([
                GATLayer(hidden_dim, nheads, dropout=dropout) for _ in range(num_layers)])

        self.classify_substations = nn.Linear(hidden_dim, 1) # a binary classifier on each substation
        

    def forward(self, x, adj):
        print(x.shape)
        x = self.linear(x)
        for layer in self.gat_layers:
            node_embeddings = layer(x, adj)
        substation_embeddings = pool_per_substation(node_embeddings, sub_id_to_elem_id)
        
        substation_prob_distr = F.softmax(self.classify_substations(substation_embeddings), dim = 1)

        return substation_prob_distr, node_embeddings, substation_embeddings

class NodeModelNodeLevel(nn.Module):
    
    def __init__(self, hidden_dim, nheads, num_layers = 2, dropout=0):
        """
        Predicts on which bar the node should be changed

        Args:
            num_features (int): Number of features of each node.
            hidden_dim ([int]): Number of features of each node after transformation.
            nheads (int): number of attention heads.
            num_layers (int, optional): Number of GAT layers. Defaults to 6.
            dropout (int, optional): Dropout probability. Defaults to 0.
        """
        super(NodeModelNodeLevel, self).__init__()


        self.gat_layers = nn.ModuleList([
                GATLayer(hidden_dim, nheads, dropout=dropout) for _ in range(num_layers)])
        self.classify_nodes = nn.Linear(hidden_dim, 1) # a binary classifier on each node

    
    def forward(self, x, adj = None):

        if adj is None:
              adj = torch.ones(size = (x.shape[0], x.shape[1], x.shape[1])) # fully_connecnted_adj, pays attention to padding (wrong)

        for layer in self.gat_layers:
            node_embeddings = layer(x,adj)

        # return node_embeddings
        node_prob_distr = F.softmax(self.classify_nodes(node_embeddings), dim = 1)
        
        return node_prob_distr



class StepwiseGNNModel(nn.Module):

    def __init__(self, substaion_model, node_model):
        """
        Puts the Substation and Node models together.
        """
        super(StepwiseGNNModel, self).__init__()

        self.substaion_model = substaion_model
        self.node_model = node_model

    def forward(self,x, adj):
        substation_prob_distr, node_embeddings, substation_embeddings = self.substaion_model(x, adj)

        sampler = torch.distributions.Categorical(probs = substation_prob_distr.flatten())
        sampled_sub = sampler.sample().item()
        print(f"Sampled substation {sampled_sub} with the following nodes {sub_id_to_elem_id[sampled_sub]}")

        subset_node_embeddings_idx = sub_id_to_elem_id[sampled_sub]
        subset_node_embeddings = node_embeddings[:, subset_node_embeddings_idx, :]

        substation_embeddings.shape, subset_node_embeddings.shape

        context_embedding = torch.mean(substation_embeddings, dim = 1, keepdim=True).expand(subset_node_embeddings.shape)
        enriched_node_embedding = torch.cat((context_embedding, subset_node_embeddings), dim = 2)

        node_prob_distr = self.node_model(enriched_node_embedding)

        return node_prob_distr

In [38]:
from grid2op_env.medha_action_space import create_action_space, remove_redundant_actions

In [39]:
class NodeModelLinkPrediction(nn.Module):

    def __init__(self):
        super(NodeModelLinkPrediction. self).__init__()

    def forward(self,x):
        pass

In [40]:
num_features = 7
hidden_dim = 128
nheads = 1
num_layers_substation = 6
num_layers_node = 3



enc = SubstationModel(num_features = num_features, hidden_dim = hidden_dim, nheads = nheads)
dec = NodeModelNodeLevel(hidden_dim = 2*hidden_dim, nheads = nheads)

the_model = StepwiseGNNModel(enc, dec)


### Testing the encoder

In [41]:
obs = rllib_env.reset()
substation_prob_distr, node_embeddings, substation_embeddings = enc(vectorize_obs(obs, env).unsqueeze(0), torch.from_numpy(obs["connectivity_matrix"]))


torch.Size([1, 56, 7])
after softamx tensor([[[0.2246, 0.2303, 0.3004,  ..., 0.0000, 0.0000, 0.0000],
         [0.2232, 0.2288, 0.2979,  ..., 0.0000, 0.0000, 0.0000],
         [0.3227, 0.3283, 0.3490,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.1977, 0.2472, 0.3436],
         [0.0000, 0.0000, 0.0000,  ..., 0.1994, 0.2377, 0.3269],
         [0.0000, 0.0000, 0.0000,  ..., 0.2574, 0.3210, 0.4216]]],
       grad_fn=<IndexPutBackward0>)
after matmul tensor([[[-1.2392, -0.1989,  1.5650,  ...,  0.9723,  0.1651, -1.0575],
         [-1.2334, -0.1969,  1.5692,  ...,  0.9766,  0.1711, -1.0626],
         [-1.2872, -0.1671,  1.6245,  ...,  0.9633,  0.2025, -1.0490],
         ...,
         [-1.4649, -0.2313,  1.4603,  ...,  0.8024, -0.0048, -0.8586],
         [-1.4920, -0.2010,  1.5310,  ...,  0.8155,  0.0460, -0.8756],
         [-1.4941, -0.1792,  1.5706,  ...,  0.8241,  0.0830, -0.8870]]],
       grad_fn=<UnsafeViewBackward0>)
output here tensor([[[-1.239

### Putting encoder and decoder together

In [42]:
sampler = torch.distributions.Categorical(probs = substation_prob_distr.flatten())
sampled_sub = sampler.sample().item()

subset_node_embeddings_idx = sub_id_to_elem_id[sampled_sub]
subset_node_embeddings = node_embeddings[:, subset_node_embeddings_idx, :]

substation_embeddings.shape, subset_node_embeddings.shape

context_embedding = torch.mean(substation_embeddings, dim = 1, keepdim=True).expand(subset_node_embeddings.shape)
enriched_node_embedding = torch.cat((context_embedding, subset_node_embeddings), dim = 2)

enriched_node_embedding.shape

torch.Size([1, 3, 256])

In [43]:
substation_prob_distr

tensor([[[0.0718],
         [0.0718],
         [0.0720],
         [0.0717],
         [0.0717],
         [0.0707],
         [0.0711],
         [0.0718],
         [0.0714],
         [0.0716],
         [0.0712],
         [0.0715],
         [0.0705],
         [0.0711]]], grad_fn=<SoftmaxBackward0>)

In [44]:
context_embedding.shape, subset_node_embeddings.shape

context_embedding.expand(subset_node_embeddings.shape).shape

torch.Size([1, 3, 128])

## Mapping the probabilities to actions

In [45]:
obs = rllib_env.reset()
substation_prob_distr, node_embeddings, substation_embeddings = enc(vectorize_obs(obs, env).unsqueeze(0), torch.from_numpy(obs["connectivity_matrix"]))


torch.Size([1, 56, 7])
after softamx tensor([[[0.2268, 0.2305, 0.2988,  ..., 0.0000, 0.0000, 0.0000],
         [0.2246, 0.2283, 0.2975,  ..., 0.0000, 0.0000, 0.0000],
         [0.3232, 0.3285, 0.3483,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.1981, 0.2470, 0.3422],
         [0.0000, 0.0000, 0.0000,  ..., 0.1979, 0.2384, 0.3242],
         [0.0000, 0.0000, 0.0000,  ..., 0.2600, 0.3196, 0.4204]]],
       grad_fn=<IndexPutBackward0>)
after matmul tensor([[[-1.2389, -0.1980,  1.5673,  ...,  0.9734,  0.1671, -1.0588],
         [-1.2282, -0.1987,  1.5658,  ...,  0.9785,  0.1699, -1.0646],
         [-1.2800, -0.1697,  1.6199,  ...,  0.9659,  0.2009, -1.0519],
         ...,
         [-1.4576, -0.2269,  1.4743,  ...,  0.8134,  0.0090, -0.8715],
         [-1.4738, -0.2007,  1.5372,  ...,  0.8300,  0.0568, -0.8924],
         [-1.4890, -0.1743,  1.5853,  ...,  0.8339,  0.0968, -0.8987]]],
       grad_fn=<UnsafeViewBackward0>)
output here tensor([[[-1.238

In [46]:
subset_node_embeddings_idx = sub_id_to_elem_id[sampled_sub]
subset_node_embeddings = node_embeddings[:, subset_node_embeddings_idx, :]

substation_embeddings.shape, subset_node_embeddings.shape

context_embedding = torch.mean(substation_embeddings, dim = 1, keepdim=True).expand(subset_node_embeddings.shape)
enriched_node_embedding = torch.cat((context_embedding, subset_node_embeddings), dim = 2)

enriched_node_embedding.shape

torch.Size([1, 3, 256])

In [47]:
batch_substaions = []
lengths = []
for subset_node_embeddings_idx in sub_id_to_elem_id.values():
    #print(subset_node_embeddings_idx)
    subset_node_embeddings = node_embeddings[:, subset_node_embeddings_idx, :]
    context_embedding = torch.mean(substation_embeddings, dim = 1, keepdim=True).expand(subset_node_embeddings.shape)
    enriched_node_embedding = torch.cat((context_embedding, subset_node_embeddings), dim = 2)
    
    batch_substaions.append(enriched_node_embedding.squeeze(0))
    lengths.append(len(subset_node_embeddings_idx))
    #print(enriched_node_embedding.shape)

padded = pad_sequence(batch_substaions, batch_first=True)#.shape

adj_matrices = torch.zeros(padded.shape[0], padded.shape[1], padded.shape[1])
for i, lentgh in enumerate(lengths):
    adj_matrices[i, :lentgh, :lentgh] = 1

#adj_matrices = adj_matrices.masked_fill(adj_matrices==0, -np.inf)
adj_matrices
#dec(padded)
#lenghts   
out = dec(padded, adj_matrices)

after softamx tensor([[[0.2292, 0.3924, 0.3785, 0.0000, 0.0000, 0.0000],
         [0.2418, 0.3863, 0.3719, 0.0000, 0.0000, 0.0000],
         [0.2414, 0.3864, 0.3722, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.1488, 0.1379, 0.1640, 0.1934, 0.1825, 0.1734],
         [0.1485, 0.1373, 0.1639, 0.1942, 0.1830, 0.1731],
         [0.1470, 0.1368, 0.1642, 0.1938, 0.1825, 0.1757],
         [0.1465, 0.1372, 0.1645, 0.1927, 0.1818, 0.1773],
         [0.1468, 0.1372, 0.1644, 0.1930, 0.1820, 0.1766],
         [0.1458, 0.1359, 0.1643, 0.1943, 0.1827, 0.1770]],

        [[0.2430, 0.2142, 0.2759, 0.2669, 0.0000, 0.0000],
         [0.2431, 0.2123, 0.2775, 0.2671, 0.0000, 0.0000],
         [0.2418, 0.2154, 0.2747, 0.2681, 0.0000, 0.0000],
         [0.2414, 0.2142, 0.2755, 0.2689, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.00

In [48]:
padded

tensor([[[ 0.2530,  0.1137,  0.0658,  ..., -0.0908, -0.1273,  0.2096],
         [ 0.2530,  0.1137,  0.0658,  ..., -0.1451, -0.0122,  0.2563],
         [ 0.2530,  0.1137,  0.0658,  ..., -0.1530, -0.0138,  0.2625],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.2530,  0.1137,  0.0658,  ..., -0.1925, -0.0374,  0.2897],
         [ 0.2530,  0.1137,  0.0658,  ..., -0.1616, -0.0650,  0.2645],
         [ 0.2530,  0.1137,  0.0658,  ..., -0.1387, -0.0457,  0.2477],
         [ 0.2530,  0.1137,  0.0658,  ..., -0.1559, -0.0080,  0.2637],
         [ 0.2530,  0.1137,  0.0658,  ..., -0.1569, -0.0175,  0.2639],
         [ 0.2530,  0.1137,  0.0658,  ..., -0.0894, -0.0631,  0.2087]],

        [[ 0.2530,  0.1137,  0.0658,  ..., -0.1794, -0.0479,  0.2787],
         [ 0.2530,  0.1137,  0.0658,  ..., -0.1259, -0.0961,  0.2354],
  

In [24]:
a =F.softmax(adj_matrices + 1e-15, dim = 2)
torch.sum(a, dim = 0).shape

torch.Size([6, 6])

In [25]:
a[0,0].sum()

tensor(1.)

In [26]:
a

tensor([[[0.2437, 0.2437, 0.2437, 0.0896, 0.0896, 0.0896],
         [0.2437, 0.2437, 0.2437, 0.0896, 0.0896, 0.0896],
         [0.2437, 0.2437, 0.2437, 0.0896, 0.0896, 0.0896],
         [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
         [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
         [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667]],

        [[0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
         [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
         [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
         [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
         [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
         [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667]],

        [[0.2112, 0.2112, 0.2112, 0.2112, 0.0777, 0.0777],
         [0.2112, 0.2112, 0.2112, 0.2112, 0.0777, 0.0777],
         [0.2112, 0.2112, 0.2112, 0.2112, 0.0777, 0.0777],
         [0.2112, 0.2112, 0.2112, 0.2112, 0.0777, 0.0777],
         [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.

In [27]:
a[0].sum()

tensor(6.)

Exception ignored in: <function GymEnv.__del__ at 0x7f8278b61170>
Traceback (most recent call last):
  File "/opt/miniconda3/envs/rlib_grid/lib/python3.7/site-packages/grid2op/gym_compat/gymenv.py", line 81, in __del__
    self.close()
  File "/opt/miniconda3/envs/rlib_grid/lib/python3.7/site-packages/grid2op/gym_compat/gymenv.py", line 72, in close
    self.action_space.close()
AttributeError: 'CustomDiscreteActions' object has no attribute 'close'


In [122]:
dec = NodeModelNodeLevel(num_features = 7, hidden_dim = 128, nheads = 4)

TypeError: __init__() got an unexpected keyword argument 'num_features'

In [143]:
import numpy as np
torch.mean(out[:, [0,1,2], : ], 1, keepdim=True).shape

torch.Size([1, 1, 128])

In [51]:
target = [[[1,2,3], 
            [2,4,5,6]], [[1,2,3], [2,4,5,6], [2,4,6,7,8]]]#[0]
max_length = max(len(row) for row in target)
print(max_length)
max_cols = max([len(row) for batch in target for row in batch])
max_rows = max([len(batch) for batch in target])
padded = [batch + [[0] * (max_cols)] * (max_rows - len(batch)) for batch in target]
padded = torch.tensor([row + [0] * (max_cols - len(row)) for batch in padded for row in batch])
padded = padded.view(-1, max_rows, max_cols)

3


In [48]:
target[0]

[[1, 2, 3], [2, 4, 5, 6]]